In [8]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
print(os.listdir("cell_images/"))
from keras.models import Sequential
ORIGIN_PATH=os.getcwd()
DATA_ORIGIN_PATH = os.path.abspath("cell_images")
TRAINING_GENERATOR_PATH = os.path.abspath("MalariaTrainData")
TESTING_GENERATOR_PATH = os.path.abspath("MalariaTestData")


['Parasitized', 'Uninfected']


# Data Restructuring
I couldn't find a straightforward method to separate the data into Train/Test. Keras ImageDataGenerator provides the means to splitting the data into training/validation, yet I'd like to test the model on a data that it hasn't seen before (not even through validation)

The following few celles creates 2 folders, containing Training data and Testing data. The images change randomly each run.

In [9]:
from keras.preprocessing.image import ImageDataGenerator

try :
    os.chdir(ORIGIN_PATH)
    os.mkdir("MalariaTrainData")
    os.chdir("MalariaTrainData")
    os.mkdir("Parasitized")
    os.mkdir("Uninfected")
    os.chdir(ORIGIN_PATH)
    os.mkdir("MalariaTestData")
    os.chdir("MalariaTestData")
    os.mkdir("Parasitized")
    os.mkdir("Uninfected")
    os.chdir(ORIGIN_PATH)

except :
    pass

In [10]:
test_precentage = 0.3


destination_1_path_train = os.path.join(ORIGIN_PATH,"MalariaTrainData","Parasitized")
destination_0_path_train = os.path.join(ORIGIN_PATH,"MalariaTrainData","Uninfected")
destination_1_path_test = os.path.join(ORIGIN_PATH,"MalariaTestData","Parasitized")
destination_0_path_test = os.path.join(ORIGIN_PATH,"MalariaTestData","Uninfected")
source_1_path = os.path.join(DATA_ORIGIN_PATH,"Parasitized")
source_0_path = os.path.join(DATA_ORIGIN_PATH,"Uninfected")
import random
from shutil import copyfile

destination_1_path=""
destination_0_path=""

os.chdir(source_1_path)
imgs = os.listdir()
total = len(imgs)
progress=0.0
for img in imgs:
    progress+=1
    file_path = os.path.join(source_1_path,img)
    if random.random() > (1-test_precentage):
        destination_1_path = destination_1_path_test
    else :
        destination_1_path = destination_1_path_train
        
    dest_path = os.path.join(destination_1_path,img)
    copyfile(file_path,dest_path)
    print('%.3f'%(100*progress/total))
    
os.chdir(source_0_path)
imgs = os.listdir()
total = len(imgs)
progress=0.0
for img in imgs:
    progress+=1
    file_path = os.path.join(source_0_path,img)
    if random.random() > (1-test_precentage):
        destination_0_path = destination_0_path_test
    else :
        destination_0_path = destination_0_path_train
        
    dest_path = os.path.join(destination_0_path,img)
    copyfile(file_path,dest_path)
    print('%.3f'%(100*progress/total))
os.chdir(ORIGIN_PATH)

0.007
0.015
0.022
0.029
0.036
0.044
0.051
0.058
0.065
0.073
0.080
0.087
0.094
0.102
0.109
0.116
0.123
0.131
0.138
0.145
0.152
0.160
0.167
0.174
0.181
0.189
0.196
0.203
0.210
0.218
0.225
0.232
0.239
0.247
0.254
0.261
0.269
0.276
0.283
0.290
0.298
0.305
0.312
0.319
0.327
0.334
0.341
0.348
0.356
0.363
0.370
0.377
0.385
0.392
0.399
0.406
0.414
0.421
0.428
0.435
0.443
0.450
0.457
0.464
0.472
0.479
0.486
0.493
0.501
0.508
0.515
0.522
0.530
0.537
0.544
0.552
0.559
0.566
0.573
0.581
0.588
0.595
0.602
0.610
0.617
0.624
0.631
0.639
0.646
0.653
0.660
0.668
0.675
0.682
0.689
0.697
0.704
0.711
0.718
0.726
0.733
0.740
0.747
0.755
0.762
0.769
0.776
0.784
0.791
0.798
0.806
0.813
0.820
0.827
0.835
0.842
0.849
0.856
0.864
0.871
0.878
0.885
0.893
0.900
0.907
0.914
0.922
0.929
0.936
0.943
0.951
0.958
0.965
0.972
0.980
0.987
0.994
1.001
1.009
1.016
1.023
1.030
1.038
1.045
1.052
1.060
1.067
1.074
1.081
1.089
1.096
1.103
1.110
1.118
1.125
1.132
1.139
1.147
1.154
1.161
1.168
1.176
1.183
1.190
1.197
1.205
1.21

9.956
9.964
9.971
9.978
9.985
9.993
10.000
10.007
10.015
10.022
10.029
10.036
10.044
10.051
10.058
10.065
10.073
10.080
10.087
10.094
10.102
10.109
10.116
10.123
10.131
10.138
10.145
10.152
10.160
10.167
10.174
10.181
10.189
10.196
10.203
10.210
10.218
10.225
10.232
10.239
10.247
10.254
10.261
10.269
10.276
10.283
10.290
10.298
10.305
10.312
10.319
10.327
10.334
10.341
10.348
10.356
10.363
10.370
10.377
10.385
10.392
10.399
10.406
10.414
10.421
10.428
10.435
10.443
10.450
10.457
10.464
10.472
10.479
10.486
10.493
10.501
10.508
10.515
10.522
10.530
10.537
10.544
10.552
10.559
10.566
10.573
10.581
10.588
10.595
10.602
10.610
10.617
10.624
10.631
10.639
10.646
10.653
10.660
10.668
10.675
10.682
10.689
10.697
10.704
10.711
10.718
10.726
10.733
10.740
10.747
10.755
10.762
10.769
10.776
10.784
10.791
10.798
10.806
10.813
10.820
10.827
10.835
10.842
10.849
10.856
10.864
10.871
10.878
10.885
10.893
10.900
10.907
10.914
10.922
10.929
10.936
10.943
10.951
10.958
10.965
10.972
10.980
10.987
10.99

18.752
18.759
18.766
18.774
18.781
18.788
18.795
18.803
18.810
18.817
18.824
18.832
18.839
18.846
18.853
18.861
18.868
18.875
18.882
18.890
18.897
18.904
18.911
18.919
18.926
18.933
18.940
18.948
18.955
18.962
18.970
18.977
18.984
18.991
18.999
19.006
19.013
19.020
19.028
19.035
19.042
19.049
19.057
19.064
19.071
19.078
19.086
19.093
19.100
19.107
19.115
19.122
19.129
19.136
19.144
19.151
19.158
19.165
19.173
19.180
19.187
19.194
19.202
19.209
19.216
19.224
19.231
19.238
19.245
19.253
19.260
19.267
19.274
19.282
19.289
19.296
19.303
19.311
19.318
19.325
19.332
19.340
19.347
19.354
19.361
19.369
19.376
19.383
19.390
19.398
19.405
19.412
19.419
19.427
19.434
19.441
19.448
19.456
19.463
19.470
19.478
19.485
19.492
19.499
19.507
19.514
19.521
19.528
19.536
19.543
19.550
19.557
19.565
19.572
19.579
19.586
19.594
19.601
19.608
19.615
19.623
19.630
19.637
19.644
19.652
19.659
19.666
19.673
19.681
19.688
19.695
19.702
19.710
19.717
19.724
19.731
19.739
19.746
19.753
19.761
19.768
19.775
19.782

27.612
27.620
27.627
27.634
27.642
27.649
27.656
27.663
27.671
27.678
27.685
27.692
27.700
27.707
27.714
27.721
27.729
27.736
27.743
27.750
27.758
27.765
27.772
27.779
27.787
27.794
27.801
27.808
27.816
27.823
27.830
27.837
27.845
27.852
27.859
27.866
27.874
27.881
27.888
27.896
27.903
27.910
27.917
27.925
27.932
27.939
27.946
27.954
27.961
27.968
27.975
27.983
27.990
27.997
28.004
28.012
28.019
28.026
28.033
28.041
28.048
28.055
28.062
28.070
28.077
28.084
28.091
28.099
28.106
28.113
28.120
28.128
28.135
28.142
28.149
28.157
28.164
28.171
28.179
28.186
28.193
28.200
28.208
28.215
28.222
28.229
28.237
28.244
28.251
28.258
28.266
28.273
28.280
28.287
28.295
28.302
28.309
28.316
28.324
28.331
28.338
28.345
28.353
28.360
28.367
28.374
28.382
28.389
28.396
28.403
28.411
28.418
28.425
28.433
28.440
28.447
28.454
28.462
28.469
28.476
28.483
28.491
28.498
28.505
28.512
28.520
28.527
28.534
28.541
28.549
28.556
28.563
28.570
28.578
28.585
28.592
28.599
28.607
28.614
28.621
28.628
28.636
28.643

36.343
36.350
36.357
36.364
36.372
36.379
36.386
36.393
36.401
36.408
36.415
36.422
36.430
36.437
36.444
36.451
36.459
36.466
36.473
36.480
36.488
36.495
36.502
36.509
36.517
36.524
36.531
36.538
36.546
36.553
36.560
36.567
36.575
36.582
36.589
36.597
36.604
36.611
36.618
36.626
36.633
36.640
36.647
36.655
36.662
36.669
36.676
36.684
36.691
36.698
36.705
36.713
36.720
36.727
36.734
36.742
36.749
36.756
36.763
36.771
36.778
36.785
36.792
36.800
36.807
36.814
36.821
36.829
36.836
36.843
36.851
36.858
36.865
36.872
36.880
36.887
36.894
36.901
36.909
36.916
36.923
36.930
36.938
36.945
36.952
36.959
36.967
36.974
36.981
36.988
36.996
37.003
37.010
37.017
37.025
37.032
37.039
37.046
37.054
37.061
37.068
37.075
37.083
37.090
37.097
37.104
37.112
37.119
37.126
37.134
37.141
37.148
37.155
37.163
37.170
37.177
37.184
37.192
37.199
37.206
37.213
37.221
37.228
37.235
37.242
37.250
37.257
37.264
37.271
37.279
37.286
37.293
37.300
37.308
37.315
37.322
37.329
37.337
37.344
37.351
37.358
37.366
37.373

...

60.929
60.936
60.943
60.951
60.958
60.965
60.972
60.980
60.987
60.994
61.001
61.009
61.016
61.023
61.030
61.038
61.045
61.052
61.060
61.067
61.074
61.081
61.089
61.096
61.103
61.110
61.118
61.125
61.132
61.139
61.147
61.154
61.161
61.168
61.176
61.183
61.190
61.197
61.205
61.212
61.219
61.226
61.234
61.241
61.248
61.255
61.263
61.270
61.277
61.284
61.292
61.299
61.306
61.313
61.321
61.328
61.335
61.343
61.350
61.357
61.364
61.372
61.379
61.386
61.393
61.401
61.408
61.415
61.422
61.430
61.437
61.444
61.451
61.459
61.466
61.473
61.480
61.488
61.495
61.502
61.509
61.517
61.524
61.531
61.538
61.546
61.553
61.560
61.567
61.575
61.582
61.589
61.597
61.604
61.611
61.618
61.626
61.633
61.640
61.647
61.655
61.662
61.669
61.676
61.684
61.691
61.698
61.705
61.713
61.720
61.727
61.734
61.742
61.749
61.756
61.763
61.771
61.778
61.785
61.792
61.800
61.807
61.814
61.821
61.829
61.836
61.843
61.851
61.858
61.865
61.872
61.880
61.887
61.894
61.901
61.909
61.916
61.923
61.930
61.938
61.945
61.952
61.959

69.855
69.862
69.869
69.877
69.884
69.891
69.898
69.906
69.913
69.920
69.927
69.935
69.942
69.949
69.956
69.964
69.971
69.978
69.985
69.993
70.000
70.007
70.015
70.022
70.029
70.036
70.044
70.051
70.058
70.065
70.073
70.080
70.087
70.094
70.102
70.109
70.116
70.123
70.131
70.138
70.145
70.152
70.160
70.167
70.174
70.181
70.189
70.196
70.203
70.210
70.218
70.225
70.232
70.239
70.247
70.254
70.261
70.269
70.276
70.283
70.290
70.298
70.305
70.312
70.319
70.327
70.334
70.341
70.348
70.356
70.363
70.370
70.377
70.385
70.392
70.399
70.406
70.414
70.421
70.428
70.435
70.443
70.450
70.457
70.464
70.472
70.479
70.486
70.493
70.501
70.508
70.515
70.522
70.530
70.537
70.544
70.552
70.559
70.566
70.573
70.581
70.588
70.595
70.602
70.610
70.617
70.624
70.631
70.639
70.646
70.653
70.660
70.668
70.675
70.682
70.689
70.697
70.704
70.711
70.718
70.726
70.733
70.740
70.747
70.755
70.762
70.769
70.776
70.784
70.791
70.798
70.806
70.813
70.820
70.827
70.835
70.842
70.849
70.856
70.864
70.871
70.878
70.885

78.752
78.759
78.766
78.774
78.781
78.788
78.795
78.803
78.810
78.817
78.824
78.832
78.839
78.846
78.853
78.861
78.868
78.875
78.882
78.890
78.897
78.904
78.911
78.919
78.926
78.933
78.940
78.948
78.955
78.962
78.970
78.977
78.984
78.991
78.999
79.006
79.013
79.020
79.028
79.035
79.042
79.049
79.057
79.064
79.071
79.078
79.086
79.093
79.100
79.107
79.115
79.122
79.129
79.136
79.144
79.151
79.158
79.165
79.173
79.180
79.187
79.194
79.202
79.209
79.216
79.224
79.231
79.238
79.245
79.253
79.260
79.267
79.274
79.282
79.289
79.296
79.303
79.311
79.318
79.325
79.332
79.340
79.347
79.354
79.361
79.369
79.376
79.383
79.390
79.398
79.405
79.412
79.419
79.427
79.434
79.441
79.448
79.456
79.463
79.470
79.478
79.485
79.492
79.499
79.507
79.514
79.521
79.528
79.536
79.543
79.550
79.557
79.565
79.572
79.579
79.586
79.594
79.601
79.608
79.615
79.623
79.630
79.637
79.644
79.652
79.659
79.666
79.673
79.681
79.688
79.695
79.702
79.710
79.717
79.724
79.731
79.739
79.746
79.753
79.761
79.768
79.775
79.782

87.271
87.279
87.286
87.293
87.300
87.308
87.315
87.322
87.329
87.337
87.344
87.351
87.358
87.366
87.373
87.380
87.388
87.395
87.402
87.409
87.417
87.424
87.431
87.438
87.446
87.453
87.460
87.467
87.475
87.482
87.489
87.496
87.504
87.511
87.518
87.525
87.533
87.540
87.547
87.554
87.562
87.569
87.576
87.583
87.591
87.598
87.605
87.612
87.620
87.627
87.634
87.642
87.649
87.656
87.663
87.671
87.678
87.685
87.692
87.700
87.707
87.714
87.721
87.729
87.736
87.743
87.750
87.758
87.765
87.772
87.779
87.787
87.794
87.801
87.808
87.816
87.823
87.830
87.837
87.845
87.852
87.859
87.866
87.874
87.881
87.888
87.896
87.903
87.910
87.917
87.925
87.932
87.939
87.946
87.954
87.961
87.968
87.975
87.983
87.990
87.997
88.004
88.012
88.019
88.026
88.033
88.041
88.048
88.055
88.062
88.070
88.077
88.084
88.091
88.099
88.106
88.113
88.120
88.128
88.135
88.142
88.149
88.157
88.164
88.171
88.179
88.186
88.193
88.200
88.208
88.215
88.222
88.229
88.237
88.244
88.251
88.258
88.266
88.273
88.280
88.287
88.295
88.302

96.110
96.118
96.125
96.132
96.139
96.147
96.154
96.161
96.168
96.176
96.183
96.190
96.197
96.205
96.212
96.219
96.226
96.234
96.241
96.248
96.255
96.263
96.270
96.277
96.284
96.292
96.299
96.306
96.313
96.321
96.328
96.335
96.343
96.350
96.357
96.364
96.372
96.379
96.386
96.393
96.401
96.408
96.415
96.422
96.430
96.437
96.444
96.451
96.459
96.466
96.473
96.480
96.488
96.495
96.502
96.509
96.517
96.524
96.531
96.538
96.546
96.553
96.560
96.567
96.575
96.582
96.589
96.597
96.604
96.611
96.618
96.626
96.633
96.640
96.647
96.655
96.662
96.669
96.676
96.684
96.691
96.698
96.705
96.713
96.720
96.727
96.734
96.742
96.749
96.756
96.763
96.771
96.778
96.785
96.792
96.800
96.807
96.814
96.821
96.829
96.836
96.843
96.851
96.858
96.865
96.872
96.880
96.887
96.894
96.901
96.909
96.916
96.923
96.930
96.938
96.945
96.952
96.959
96.967
96.974
96.981
96.988
96.996
97.003
97.010
97.017
97.025
97.032
97.039
97.046
97.054
97.061
97.068
97.075
97.083
97.090
97.097
97.104
97.112
97.119
97.126
97.134
97.141

## Building Data Generators

In [11]:
from keras.preprocessing.image import ImageDataGenerator

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   validation_split=0.4,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_set = train_datagen.flow_from_directory(TRAINING_GENERATOR_PATH,
                                                 target_size = (64, 64),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='training')
validation_set = train_datagen.flow_from_directory(TRAINING_GENERATOR_PATH,
                                                 target_size = (64, 64),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='validation')
test_set = train_datagen.flow_from_directory(TESTING_GENERATOR_PATH,
                                                 target_size = (64, 64),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='validation')

Found 11568 images belonging to 2 classes.
Found 7710 images belonging to 2 classes.
Found 3312 images belonging to 2 classes.


# Modeling

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD

classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate=0.5))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# # Compiling the CNN

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Fitting the classifier

**WARNING** : Takes several hours to execute

In [12]:
classifier.fit_generator(train_set,
                           steps_per_epoch=363,
                           epochs=10,
                           validation_data=validation_set,
                           validation_steps=242)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
363/363 [==============================] - 51s 139ms/step - loss: 0.5440 - acc: 0.7416 - val_loss: 0.3721 - val_acc: 0.8589
Epoch 2/10
363/363 [==============================] - 47s 130ms/step - loss: 0.2922 - acc: 0.8986 - val_loss: 0.3079 - val_acc: 0.9084
Epoch 3/10
363/363 [==============================] - 47s 128ms/step - loss: 0.2322 - acc: 0.9221 - val_loss: 0.2204 - val_acc: 0.8897
Epoch 4/10
363/363 [==============================] - 50s 137ms/step - loss: 0.2222 - acc: 0.9172 - val_loss: 0.1970 - val_acc: 0.9097
Epoch 5/10
363/363 [==============================] - 51s 142ms/step - loss: 0.2090 - acc: 0.9299 - val_loss: 0.2196 - val_acc: 0.8893
Epoch 6/10
363/363 [==============================] - 52s 144ms/step - loss: 0.1955 - acc: 0.9347 - val_loss: 0.1842 - val_acc: 0.9283
Epoch 7/10
363/363 [==============================] - 48s 134ms/step - loss: 0.1897 - acc: 0.9397 - val_loss: 0.1952 - val_acc: 0.9192
Epoch 8

saving the model

In [33]:
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
classifier.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


Loading

In [34]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


## Evaluating the model on the test set
Testing the model on a data that it hasn't seen before

In [39]:
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate_generator(generator=test_set,steps=102)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

acc: 94.45%


# Using a pre-built model

In [14]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   validation_split=0.4,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_set = train_datagen.flow_from_directory(TRAINING_GENERATOR_PATH,
                                                 target_size = (224,224),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='training')
validation_set = train_datagen.flow_from_directory(TRAINING_GENERATOR_PATH,
                                                 target_size = (224,224),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='validation')
test_set = train_datagen.flow_from_directory(TESTING_GENERATOR_PATH,
                                                 target_size = (224,224),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='validation')

Found 11568 images belonging to 2 classes.
Found 7710 images belonging to 2 classes.
Found 3312 images belonging to 2 classes.


In [1]:
import keras
vgg16_model = keras.applications.vgg16.VGG16()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
from keras.models import Sequential
model = Sequential()
i = 0
for layer in vgg16_model.layers:
    i +=1
    if i < 23 :
        model.add(layer)
        
for layer in model.layers:
    layer.trainable=False
    
    
from keras.layers import Dense
model.add(Dense(1,activation = 'softmax'))

from keras.optimizers import Adam
model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
model.fit_generator(train_set,
                           steps_per_epoch=363,
                           epochs=10,
                           validation_data=validation_set,
                           validation_steps=242,
                           verbose=1)


Epoch 1/10
363/363 [==============================] - 6133s 17s/step - loss: 7.9657 - acc: 0.5003 - val_loss: 7.9650 - val_acc: 0.5004
Epoch 2/10
363/363 [==============================] - 6150s 17s/step - loss: 7.9492 - acc: 0.5014 - val_loss: 7.9630 - val_acc: 0.5005
Epoch 3/10
363/363 [==============================] - 6160s 17s/step - loss: 7.9671 - acc: 0.5003 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 4/10
363/363 [==============================] - 6151s 17s/step - loss: 7.9822 - acc: 0.4993 - val_loss: 7.9444 - val_acc: 0.5017
Epoch 5/10
363/363 [==============================] - 6166s 17s/step - loss: 7.9506 - acc: 0.5013 - val_loss: 7.9774 - val_acc: 0.4996
Epoch 6/10
363/363 [==============================] - 6137s 17s/step - loss: 7.9602 - acc: 0.5007 - val_loss: 7.9609 - val_acc: 0.5006
Epoch 7/10
363/363 [==============================] - 6133s 17s/step - loss: 7.9657 - acc: 0.5003 - val_loss: 7.9485 - val_acc: 0.5014
Epoch 8/10
363/363 [==============================] - 6

## Evaluating the model on the test set

In [17]:
score = model.evaluate_generator(generator=test_set,steps=102, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

102/102 [==============================] - 1038s 10s/step
acc: 50.15%


# Summary

- The manually built model performed well, with an accuracy of 94.45% on the test set
- The prebuilt model used (vgg16) didn't fit well, the final accuracy on the test set is ~50%, it'll be a good idea to use a different one, but as long as the manually constructed moel performed sufficiently well, there's no need for that